<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/JARVIS_LAMMPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
# %env LD_PRELOAD=
# !apt install cmake build-essential git ccache
# Clone the LAMMPS git
if not os.path.exists('lammps'):
   os.system('git clone https://github.com/lammps/lammps.git')


env: LD_PRELOAD=
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
ccache is already the newest version (3.4.1-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
git is already the newest version (1:2.17.1-1ubuntu0.11).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [17]:
os.chdir('lammps/src')


In [20]:
!make yes-MANYBODY
!make serial

Installing package MANYBODY
make[1]: Entering directory '/content/lammps/src/STUBS'
make[1]: 'libmpi_stubs.a' is up to date.
make[1]: Leaving directory '/content/lammps/src/STUBS'
Gathering installed package information (may take a little while)
make[1]: Entering directory '/content/lammps/src'
Gathering git version information
make[1]: Leaving directory '/content/lammps/src'
Compiling LAMMPS for machine serial
make[1]: Entering directory '/content/lammps/src/Obj_serial'
make[1]: Leaving directory '/content/lammps/src/Obj_serial'
make[1]: Entering directory '/content/lammps/src/Obj_serial'
g++ -g -O3 -std=c++11  -DLAMMPS_GZIP -DLAMMPS_MEMALIGN=64    -I../STUBS     -c ../min_cg.cpp
g++ -g -O3 -std=c++11  -DLAMMPS_GZIP -DLAMMPS_MEMALIGN=64    -I../STUBS     -c ../pair_morse.cpp
g++ -g -O3 -std=c++11  -DLAMMPS_GZIP -DLAMMPS_MEMALIGN=64    -I../STUBS     -c ../procmap.cpp
g++ -g -O3 -std=c++11  -DLAMMPS_GZIP -DLAMMPS_MEMALIGN=64    -I../STUBS     -c ../pair_bop.cpp
g++ -g -O3 -std=c++11  -

In [31]:
!export lammps_cmd="/content/lammps/src/lmp_serial"

In [37]:
!wget https://www.ctcms.nist.gov/~knc6/DOWNLOADS/Al_zhou.eam.alloy
!wget https://raw.githubusercontent.com/usnistgov/jarvis/master/jarvis/tasks/lammps/templates/inelast.mod

--2022-06-26 22:10:11--  https://www.ctcms.nist.gov/~knc6/DOWNLOADS/Al_zhou.eam.alloy
Resolving www.ctcms.nist.gov (www.ctcms.nist.gov)... 129.6.13.19, 2610:20:6005:13::19
Connecting to www.ctcms.nist.gov (www.ctcms.nist.gov)|129.6.13.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 710486 (694K) [application/zip]
Saving to: ‘Al_zhou.eam.alloy’

Al_zhou.eam.alloy   100%[===================>] 693.83K  --.-KB/s    in 0.04s   

2022-06-26 22:10:11 (19.2 MB/s) - ‘Al_zhou.eam.alloy’ saved [710486/710486]

--2022-06-26 22:10:12--  https://raw.githubusercontent.com/usnistgov/jarvis/master/jarvis/tasks/lammps/templates/inelast.mod
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6416 (6.3K) [text/plain]
Saving to: ‘inelast.mod’

inelast.mo

In [39]:
!pip install jarvis-tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 962 kB 14.3 MB/s 
     |████████████████████████████████| 325 kB 42.5 MB/s 


In [48]:
from jarvis.tasks.lammps.lammps import LammpsJob, JobFactory
from jarvis.core.atoms import Atoms
from jarvis.db.figshare import get_jid_data
from jarvis.analysis.structure.spacegroup import Spacegroup3D
tmp_dict = get_jid_data(jid="JVASP-816", dataset="dft_3d")["atoms"]
atoms = Atoms.from_dict(tmp_dict)
# Get conventional cell
spg = Spacegroup3D(atoms)
cvn_atoms = spg.conventional_standard_structure
ff = "/content/lammps/src/Al_zhou.eam.alloy"
mod = "/content/lammps/src/inelast.mod"
cmd = "/content/lammps/src/lmp_serial<in.main>out"

Obtaining 3D dataset 55k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.


In [49]:
parameters = {
    "pair_style": "eam/alloy",
    "pair_coeff": ff,
    "atom_style": "charge",
    "control_file": mod,
}
# Test LammpsJob
lmp = LammpsJob(atoms=cvn_atoms, parameters=parameters, lammps_cmd=cmd, jobname="Test").runjob()



[0.0, 0.0, 0.0]
[0.0, 2.033705, 2.033705]
[2.033705, 0.0, 2.033705]
[2.033705, 2.033705, 0.0]
[-0.007122450392313417, -0.0071224503923133595, -0.007122450392313428]
[-0.007122450392313417, 2.033704999999999, 2.0337049999999994]
[2.0337049999999994, -0.007122450392313366, 2.0337049999999994]
[2.0337049999999994, 2.033704999999999, -0.007122450392313428]


In [50]:
!ls

data	  in.main     log.lammps  potential.mod  Test
init.mod  lammps.out  out	  std_err.txt	 Test.json


In [54]:
!pip install phonopy ase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 20.9 MB/s 


In [55]:
# Test high-throughput
job_fact = JobFactory(pair_style="eam/alloy", name="my_first_lammps_run")
job_fact.all_props_eam_alloy(atoms=cvn_atoms, ff_path=ff, lammps_cmd=cmd)

element_order is empty, reading from /content/lammps/src/Test/Test/Test/ELASTIC/potential.mod
[0.0, 0.0, 0.0]
[0.0, 2.033705, 2.033705]
[2.033705, 0.0, 2.033705]
[2.033705, 2.033705, 0.0]
element_order is empty, reading from /content/lammps/src/Test/Test/Test/ELASTIC/potential.mod
[-0.007122450392313417, -0.0071224503923133595, -0.007122450392313428]
[-0.007122450392313417, 2.033704999999999, 2.0337049999999994]
[2.0337049999999994, -0.007122450392313366, 2.0337049999999994]
[2.0337049999999994, 2.033704999999999, -0.007122450392313428]
en, final_str, forces -3.58 Al4
1.0
4.081654900784626 0.0 0.0
0.0 4.081654900784625 0.0
0.0 0.0 4.081654900784626
Al
4
Cartesian
-0.007122450392313417 -0.0071224503923133595 -0.007122450392313428
-0.007122450392313417 2.033704999999999 2.0337049999999994
2.0337049999999994 -0.007122450392313366 2.0337049999999994
2.0337049999999994 2.033704999999999 -0.007122450392313428
 []
element_order is empty, reading from /content/lammps/src/Test/Test/Test/Surf-1_

In [56]:
!ls

data		 in.main	    std_err.txt      symbol-Al-Wycoff-b_Al
disp-1		 lammps.out	    Surf-1_0_0	     symbol-Al-Wycoff-b_Al.json
disp-1.json	 log.lammps	    Surf-1_0_0.json  Test
ELASTIC		 out		    Surf-1_1_0	     Test.json
ELASTIC.json	 POSCAR		    Surf-1_1_0.json
FORCE_CONSTANTS  POSCAR-Super.vasp  Surf-1_1_1
init.mod	 potential.mod	    Surf-1_1_1.json


In [60]:
from jarvis.io.lammps.outputs import parse_material_calculation_folder
fold = '/content/lammps/src/Test/Test/Test'
x = parse_material_calculation_folder(fold)

/content/lammps/src/Test/Test/Test
json_file_name symbol-Al-Wycoff-b_Al.json
json_file_path /content/lammps/src/Test/Test/Test/symbol-Al-Wycoff-b_Al
[-0.007122459117278066, -0.007122459117278009, 4.074532540882721]
[-0.007122459117278066, 4.074532540882721, -0.007122459117278078]
[-0.007122459117278066, 4.074532540882721, 4.074532540882721]
[4.074532540882721, -0.007122459117278009, -0.007122459117278078]
[4.074532540882721, -0.007122459117278009, 4.074532540882721]
[4.074532540882721, 4.074532540882721, -0.007122459117278078]
[4.074532540882721, 4.074532540882721, 4.074532540882721]
[-0.007122459117278066, 2.033707491278036, 2.0337074912780366]
[-0.007122459117278066, 2.033707491278036, 6.115362491278036]
[-0.007122459117278066, 6.115362491278036, 2.0337074912780366]
[-0.007122459117278066, 6.115362491278036, 6.115362491278036]
[4.074532540882721, 2.033707491278036, 2.0337074912780366]
[4.074532540882721, 2.033707491278036, 6.115362491278036]
[4.074532540882721, 6.115362491278036, 2.0

In [61]:
x

{'ELASTIC.json': {'elastic_tensor': '',
  'energy_per_atom': -3.57999875,
  'final_str': Al4
  1.0
  4.081654900784626 0.0 0.0
  0.0 4.081654900784625 0.0
  0.0 0.0 4.081654900784626
  Al
  4
  Cartesian
  -0.007122450392313417 -0.0071224503923133595 -0.007122450392313428
  -0.007122450392313417 2.033704999999999 2.0337049999999994
  2.0337049999999994 -0.007122450392313366 2.0337049999999994
  2.0337049999999994 2.033704999999999 -0.007122450392313428,
  'initial_str': Al4
  1.0
  4.06741 0.0 0.0
  0.0 4.06741 0.0
  0.0 0.0 4.06741
  Al
  4
  Cartesian
  0.0 0.0 0.0
  0.0 2.033705 2.033705
  2.033705 0.0 2.033705
  2.033705 2.033705 0.0,
  'pair_coeff': 'Al_zhou.eam.alloy',
  'pair_style': ' eam/alloy ',
  'system_pressure': -2.1980167000000002e-13},
 'Surf-1_0_0.json': {'elastic_tensor': '',
  'energy_per_atom': -3.5164896428571426,
  'final_str': Al28
  1.0
  4.049344444335735 0.0 0.0
  0.0 4.0493444443363815 0.0
  0.0 0.0 46.941123276833174
  Al
  28
  Cartesian
  1.028488888916061